# Main to test the output of the functions

In [1]:
import scipy.io as sio
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os 

In [5]:
def convert_annot(annot):
    corners = np.array(list(annot.keys()))
    corners_mapping = {tuple(c): idx for idx, c in enumerate(corners)}
    edges = set()
    for corner, connections in annot.items():
        idx_c = corners_mapping[tuple(corner)]
        for other_c in connections:
            idx_other_c = corners_mapping[tuple(other_c)]
            if (idx_c, idx_other_c) not in edges and (idx_other_c, idx_c) not in edges:
                edges.add((idx_c, idx_other_c))
    edges = np.array(list(edges))
    gt_data = {
        'corners': corners,
        'edges': edges
    }
    return gt_data

In [6]:
def visualize_cond_generation(positive_pixels, confs, image, save_path, gt_corners=None, prec=None, recall=None,
                              image_masks=None, edges=None, edge_confs=None):
    image = image.copy()  # get a new copy of the original image
    if confs is not None:
        viz_confs = confs

    if edges is not None:
        preds = positive_pixels.astype(int)
        # c_degrees = dict()
        c_degrees = {}
        for edge_i, edge_pair in enumerate(edges):
            # conf = (edge_confs[edge_i] * 2) - 1
            cv2.line(image, tuple(preds[edge_pair[0]]), tuple(preds[edge_pair[1]]), (255, 255 , 0), 2)
            c_degrees[edge_pair[0]] = c_degrees.setdefault(edge_pair[0], 0) + 1
            c_degrees[edge_pair[1]] = c_degrees.setdefault(edge_pair[1], 0) + 1

    for idx, c in enumerate(positive_pixels):
        if edges is not None and idx not in c_degrees:
            continue
        if confs is None:
            cv2.circle(image, (int(c[0]), int(c[1])), 3, (0, 0, 255), -1)
            cv2.putText(image, f"{idx}", (int(c[0]), int(c[1])),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 0, 255), 1, cv2.LINE_AA)
            print(f"toa do {idx}",(int(c[0]), int(c[1])))
        else:
            cv2.circle(image, (int(c[0]), int(c[1])), 3, (0, 0, 255 * viz_confs[idx]), -1)
        # if edges is not None:
        #    cv2.putText(image, '{}'.format(c_degrees[idx]), (int(c[0]), int(c[1] - 5)), cv2.FONT_HERSHEY_SIMPLEX,
        #                0.5, (255, 0, 0), 1, cv2.LINE_AA)

    if gt_corners is not None:
        for c in gt_corners:
            cv2.circle(image, (int(c[0]), int(c[1])), 3, (0, 255, 0), -1)

    if image_masks is not None:
        mask_ids = np.where(image_masks == 1)[0]
        for mask_id in mask_ids:
            y_idx = mask_id // 64
            x_idx = (mask_id - y_idx * 64)
            x_coord = x_idx * 4
            y_coord = y_idx * 4
            cv2.rectangle(image, (x_coord, y_coord), (x_coord + 3, y_coord + 3), (127, 127, 0), thickness=-1)

    return image


In [2]:
images = os.listdir('./data/')
for i, name in enumerate(images):
    if name.endswith('jpg'):
        head = name.split('.jpg')[0]
        img = cv2.imread('./data/'+head+ '.jpg')
        lines = sio.loadmat('./data/'+head+ '.mat')['lines']
        print(lines)
        points_new = {}
        for p1, p2 in lines:
            p1 = tuple(p1)
            p2 = tuple(p2)
            if p1 not in points_new:
                points_new[p1] = []
            if p2 not in points_new:
                points_new[p2] = []
            points_new[p1].append(list(p2))
            points_new[p2].append(list(p1))
        # print(points_new)
        break
    # label = convert_annot(points_new)
    # test = visualize_cond_generation(label['corners'].astype(int), None, img, None, gt_corners=label['corners'], image_masks=None, edges=label['edges'], edge_confs=None)

    # plt.imshow(test)
    # plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './data/'

In [23]:

img = cv2.imread('./data/'+'1548203615.09'+ '.jpg')
lines = sio.loadmat('./data/'+'1548203615.09'+ '.mat')['lines']
# print(lines)
points_new = {}
for p1, p2 in lines:
    p1 = tuple(p1)
    p2 = tuple(p2)
    if p1 not in points_new:
        points_new[p1] = []
    if p2 not in points_new:
        points_new[p2] = []
    points_new[p1].append(list(p2))
    points_new[p2].append(list(p1))
print(points_new)
# save points_new to npy file 
np.save('./data/'+'1548203615.09'+'.npy', points_new)


{(15.75, 155.25): [[182.5, 48.0]], (182.5, 48.0): [[15.75, 155.25], [198.5, 70.25]], (198.5, 70.25): [[182.5, 48.0], [210.0, 61.75], [222.0, 108.75], [164.5, 90.25]], (210.0, 61.75): [[198.5, 70.25]], (222.0, 108.75): [[198.5, 70.25]], (164.5, 90.25): [[198.5, 70.25]]}


# convert label

In [29]:
images = './data/rgb/'

labels = 'data/annot/'


for filename in os.listdir('./data/rgb/'):
    if filename.endswith('.jpg'):
        head = filename.split('.jpg')[0]
        img = cv2.imread(images +head+ '.jpg')
        lines = sio.loadmat(images +head+ '.mat')['lines']
        points_new = {}
        for p1, p2 in lines:
            p1 = tuple(p1)
            p2 = tuple(p2)
            if p1 not in points_new:
                points_new[p1] = []
            if p2 not in points_new:
                points_new[p2] = []
            points_new[p1].append(list(p2))
            points_new[p2].append(list(p1))
        # save points_new to npy file 
        np.save(labels+ head +'.npy', points_new)



In [2]:
base = 'data_test'
for filename in os.listdir('./data_test/rgb/'):
    if filename.endswith('.jpg'):
        filename = filename.split('.jpg')[0]
        with open(base +'/' + 'full_list'+'.txt', 'a') as f:
            f.writelines(filename+'\n')

In [3]:
import random

# Read the text file containing the paths to image files
with open(base +'/' + 'full_list.txt') as f:
    file_paths = f.readlines()

# Shuffle the list for randomization
random.shuffle(file_paths)

# Determine the lengths of the two files
total_items = len(file_paths)
train_len = int(total_items * 0.8)
test_len = total_items - train_len

# Split the list into two files
with open(base +'/'+ "train_list.txt", "w") as train_file:
    train_file.writelines(file_paths[:train_len])

with open(base +'/'+ "valid_list.txt", "w") as test_file:
    test_file.writelines(file_paths[train_len:])
